In [ ]:
import pandas as pd
from tbats import TBATS, BATS
import numpy as np

# df = pd.read_csv('entities-2023-09-02_17 33 41.csv', usecols=['start', 'mean'])
df = pd.read_csv('entities-2023-09-06_10 17 37.csv', usecols=['start', 'mean'])
df['start'] = pd.to_datetime(df['start'], format='%Y-%m-%d %H:%M:%S')
df = df.set_index('start')
df = df.asfreq(freq='H')
df = df.tail(5*7*24)

# train_period = 24 * 7 * 9
train_period = 24*5
# # for x in range(7*3):
# for x in range(1):
#     train_start = x
#     train_end = x + train_period
#     print(f"Training from {train_start} - {train_end - 1}: {df.iloc[train_start].name} - {df.iloc[train_end - 1].name}")

#     estimator = TBATS(use_box_cox=False, use_trend=False, use_damped_trend=False, seasonal_periods=[24])
#     fitted_model = estimator.fit(df[train_start:train_end]['mean'])



In [ ]:
# print(fitted_model.summary())
print(type(fitted_model))

pd.set_option('display.max_rows', None)

estimator = TBATS(seasonal_periods=[24],show_warnings=False)

for x in range(0, len(df) - train_period, 24):
# for x in range(0, 7*24*3, 24):
# for x in range(1):
    train_start = x
    train_end = x + train_period
    predict_start = df.iloc[train_end].name

    print(f"Training from {df.iloc[train_start].name} to {df.iloc[train_end - 1].name}. Predicting from {predict_start} to {predict_start + pd.Timedelta(hours=23)}")

    fitted_model = estimator.fit(df[train_start:train_end]['mean'])
    # print(fitted_model.summary())

    y_forecasted, confidence_info = fitted_model.forecast(steps=24, confidence_level=0.5)
    # print(confidence_info )
    forecasted_df = pd.DataFrame(confidence_info, index=pd.date_range(predict_start, predict_start + pd.Timedelta(hours=23), freq='H')) \
        .rename(columns={'mean': 'predicted', 'lower_bound': 'predicted_lower', 'upper_bound': 'predicted_upper'}) \
        .drop(columns=['calculated_for_level'])
    # print(forecasted_df)
    # print(df[predict_start:])
    df = df.combine_first(forecasted_df)
    
    # fitted_model = fitted_model.fit(df[:predict_start + pd.Timedelta(hours=23)]['mean'])

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from matplotlib import pyplot

pyplot.figure()
fig, ax = pyplot.subplots(figsize=(15, 5))
df[train_period-48:]['mean'].plot(ax=ax)
df[train_period-48:]['predicted'].plot(ax=ax)
# (df[train_period-48:]['mean'] - df[train_period-48:]['predicted']).plot(ax=ax)
ax.fill_between(df.index, df['predicted_lower'], df['predicted_upper'], color='k', alpha=0.1);  
pyplot.legend()
pyplot.show()

daily_sums = df[train_period:].resample('D').sum()
daily_sums = daily_sums[9:]

pyplot.figure()
fig, ax = pyplot.subplots(figsize=(15, 5))
daily_sums['mean'].plot(ax=ax)
daily_sums['predicted'].plot(ax=ax)
pyplot.legend()
pyplot.show()


print(mean_absolute_error(df[train_period:]['mean'], df[train_period:]['predicted']))
print(daily_sums)
print(mean_absolute_error(daily_sums['mean'], daily_sums['predicted']))
print(mean_squared_error(daily_sums['mean'], daily_sums['predicted']))